In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from pandas.tseries.frequencies import to_offset
from sklearn.base import BaseEstimator, TransformerMixin
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import MAE

In [ ]:
pip install neuralforecast

In [ ]:
import torch
print(torch.__version__)
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Data Preprocess

In [ ]:
def get_delivery_time(df):
    df['weekday_num'] = df['ExecutionTime'].dt.weekday
    df['ID_weekday_num'] = df['ID'].str[:3]
    df['ID_weekday_num'] = df['ID_weekday_num'].replace({'Mon':0, 'Tue':1, 'Wed':2,
                                                         'Thu':3, 'Fri':4, 'Sat':5, 'Sun':6})
    df['days_to_add'] = (df['ID_weekday_num'] - df['weekday_num'])%7
    df['delivery_date'] = df['ExecutionTime'].dt.date + pd.to_timedelta(df['days_to_add'],unit='d')
    hours = df['ID'].str[3:5].astype(int)
    minutes = ((df['ID'].str[-1].astype(int) - 1) * 15)
    df['delivery_time'] = pd.to_datetime(df['delivery_date']) + pd.to_timedelta(hours, unit='h') + pd.to_timedelta(minutes, unit='m')
    df = df.drop(columns=['weekday_num','ID_weekday_num','days_to_add','delivery_date'])

    return df


def enrich_contract_panel(df: pd.DataFrame):
    df = df.copy()
    df['delivery_time'] = pd.to_datetime(df['delivery_time'])
    df['ExecutionTime']  = pd.to_datetime(df['ExecutionTime'])
    df.sort_values(['delivery_time', 'ExecutionTime'], inplace=True)

    df['delivery_hour']  = df['delivery_time'].dt.floor('h')
    df['minute_in_hour'] = df['delivery_time'].dt.minute  # 0/15/30/45
    df['price_range'] = df['high'] - df['low']

    key = ['delivery_hour', 'ExecutionTime']
    gx  = df.groupby(key, sort=False)
    df['q2h_price_diff']  = df['close']  - gx['close'].transform('mean')
    df['q2h_volume_diff'] = df['volume'] - gx['volume'].transform('mean')

    drop_cols = ['delivery_hour','minute_in_hour']
    df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True, errors='ignore')

    return df

def merge_variables(df, df_variables):
    df_variables['wind_forecast'] = df_variables['generation_wind_onshore_forecast'] + df_variables['generation_wind_offshore_forecast']
    df_variables = df_variables.rename(columns={"generation_solar_forecast":"solar_forecast"})
    print(f"before merge: {df.shape}")
    df = df.merge(df_variables, on='delivery_time', how='left')
    print(f"after merge: {df.shape}")

    return df

def remove_outliers(df: pd.DataFrame, cols, drop_cols=('close',)):
    """
    For each col in `cols`:
      - compute the 99th percentile of absolute values
      - if col in `drop_cols`: drop rows where |col| > threshold
      - else: clip to [-threshold, +threshold]
    Returns cleaned df and thresholds dict.
    """
    df_clean = df.copy()
    before = df_clean['delivery_time'].nunique()

    mask = pd.Series(True, index=df_clean.index)

    thresholds = {}
    for col in cols:
        if not pd.api.types.is_numeric_dtype(df_clean[col]):
            continue
        thr = np.percentile(np.abs(df_clean[col].dropna()), 99)
        thresholds[col] = thr

        if col in drop_cols:
            mask &= (np.abs(df_clean[col]) <= thr)
        else:
            df_clean[col] = df_clean[col].clip(lower=-thr, upper=thr)

    df_clean = df_clean[mask]

    after = df_clean['delivery_time'].nunique()
    print(f"before remove outlier {before}")
    print(f"after remove outlier  {after}")

    return df_clean


def fill_gaps(df, full_range):
    ids = df["ID"].drop_duplicates().sort_values()
    frame_full = pd.MultiIndex.from_product(
        [ids, full_range], names=["ID", "ExecutionTime"]
    ).to_frame(index=False)
    out = frame_full.merge(df, on=["ID", "ExecutionTime"], how="left")

    return out


def add_temproal_features(df):
    df['day_of_year_sin'] = np.sin(2 * np.pi * df['delivery_time'].dt.dayofyear  / 365)
    df['day_of_year_cos'] = np.cos(2 * np.pi * df['delivery_time'].dt.dayofyear / 365)
    df['delivery_hour'] = df['delivery_time'].dt.hour
    df['delivery_hour_sin'] = np.sin(df['delivery_time'].dt.hour * (2 * np.pi / 24))
    df['delivery_hour_cos'] = np.cos(df['delivery_time'].dt.hour * (2 * np.pi / 24))
    df['delivery_weekday_sin'] = np.sin(df['delivery_time'].dt.weekday * (2 * np.pi / 7))
    df['delivery_weekday_cos'] = np.cos(df['delivery_time'].dt.weekday * (2 * np.pi / 7))
    df['delivery_month_sin'] = np.sin(df['delivery_time'].dt.month * (2 * np.pi / 12))
    df['delivery_month_cos'] = np.cos(df['delivery_time'].dt.month * (2 * np.pi / 12))
    df['time_to_expiry_h'] = (df['delivery_time'] - df['ExecutionTime']).dt.total_seconds() / 3600.0
    df['qh_sin'] = np.sin(2 * np.pi * (df['delivery_time'].dt.hour * 4 + df['delivery_time'].dt.minute // 15) / 96)
    df['qh_cos'] = np.cos(2 * np.pi * (df['delivery_time'].dt.hour * 4 + df['delivery_time'].dt.minute // 15) / 96)

    return df


def check_timestamp_completeness(df, freq=None):
    step = to_offset(freq)
    g = (df
         .drop_duplicates(['delivery_time','ExecutionTime'])
         .sort_values(['delivery_time','ExecutionTime'])
         .groupby('delivery_time', as_index=False))

    agg = g.agg(start=('ExecutionTime','min'),
                end=('ExecutionTime','max'),
                rows=('ExecutionTime','count'))

    exp = ((agg['end'] - agg['start']) / pd.to_timedelta(step.nanos, unit='ns')).astype(int) + 1
    agg['expected'] = exp
    agg['missing'] = agg['expected'] - agg['rows']
    agg['missing_ratio'] = agg['missing'] / agg['expected']
    agg['has_gap'] = agg['missing'] > 0

    return agg.sort_values('missing', ascending=False)


def filter_complete_ids(df: pd.DataFrame, completeness_df: pd.DataFrame):
    print(f"before continutiy check: {df['delivery_time'].nunique()}")
    ok_ids = completeness_df.loc[~completeness_df['has_gap'], 'delivery_time']
    df = df[df['delivery_time'].isin(ok_ids)].copy()
    print(f"after continutiy check: {df['delivery_time'].nunique()}")
    return df


def preprocess_2023(df, df_variables, train=True):
    value_cols = [c for c in df.columns if c not in ['ID', 'ExecutionTime','delivery_time','delivery_date']]
    df = (df.groupby(['ID', 'ExecutionTime'], as_index=False)[value_cols].mean())
    df = get_delivery_time(df)
    print(f"delivery_time amounts: {df['delivery_time'].nunique()}")
    # add external features:
        # 1. q2h difference on price and volume
    df = enrich_contract_panel(df)
    print(f'columns: {len(df.columns)}')
        # 2. lws forecast
        # 3. tempreture t-1d
        # 4. imported, exported forecast
    df = merge_variables(df, df_variables=df_variables)
    # remove outliers -99 percentile
    if train == True:
        df = remove_outliers(df, df.columns)
    # filter out the lack hour continutiy contract
    report = check_timestamp_completeness(df, freq="15min")
    df = filter_complete_ids(df, report)
    # add one column indentifying the war and post-covind period
    df['special_period'] = ( df['ExecutionTime'].between('2021-01-01', '2021-03-31') | df['ExecutionTime'].between('2022-02-24', '2022-12-31')).astype(int)
    return df, report


def get_last_16_steps_2024(
    df: pd.DataFrame,
    group_col: str = "delivery_time",
    time_col: str = "ExecutionTime",
    n: int = 16,
    deduplicate_times: bool = False,
    reset_index: bool = True
):

    mask_2024 = df[group_col].dt.year == 2024
    sub = df.loc[mask_2024].sort_values([group_col, time_col], kind="mergesort")

    if deduplicate_times:
        sub = sub.drop_duplicates(subset=[group_col, time_col], keep="last")

    last_mask_2024 = sub.groupby(group_col).cumcount(ascending=False) < n
    idx_last = sub.loc[last_mask_2024].index

    last = df.loc[idx_last].copy()
    rest   = df.drop(index=idx_last).copy()

    if reset_index:
        last.reset_index(drop=True, inplace=True)
        rest.reset_index(drop=True, inplace=True)

    return rest, last



def rename_df_2023(df):
    df = df.rename(columns={"ExecutionTime": "ds", "delivery_time":"unique_id", "close": "y"})
    df = df[['unique_id', 'ds', 'y', 'high', 'low', 'volume', 'price_range', 'q2h_price_diff', 'q2h_volume_diff', 'day_ahead_price',
              'load_forecast', 'solar_forecast', 'wind_forecast', 'temperature_rounded','imported', 'exported',
              'day_of_year_sin', 'day_of_year_cos' ,'delivery_month_sin',
              'delivery_month_cos','time_to_expiry_h', 'special_period', 'delivery_hour_sin', 'delivery_hour_cos', 'delivery_weekday_sin',
              'delivery_weekday_cos','qh_sin', 'qh_cos',
              'has_imported', 'has_exported', 'has_volume', 'has_q2h_volume_diff']]
    #df["has_data"] = (~df["high"].isnull()).astype(int)
    return df



In [ ]:
class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self, mad_cols=(), sparse_cols=(), passthrough_cols=()):
        self.mad_cols = list(mad_cols)
        self.sparse_cols = list(sparse_cols)  # will get indicator + arcsinh
        self.passthrough_cols = list(passthrough_cols)

        # fitted stats
        self.mad_median_ = None
        self.mad_scale_ = None
        self.sparse_median_ = None
        self.sparse_scale_ = None
        self.robust_cols_ = None
        self.robust_median_ = None
        self.robust_scale_ = None
        self.feature_names_in_ = None

    @staticmethod
    def _nanmedian(x, axis=0):
        return np.nanmedian(x, axis=axis)

    @staticmethod
    def _nanquantile(x, q, axis=0):
        return np.nanquantile(x, q, axis=axis)

    def fit(self, X, y=None):
        X = X.copy()
        self.feature_names_in_ = np.array(X.columns, dtype=object)

        numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        self.robust_cols_ = [
            c for c in numeric_cols
            if c not in self.mad_cols
            and c not in self.sparse_cols
            and c not in self.passthrough_cols
        ]

        # ---- stats for MAD columns ----
        if self.mad_cols:
            A = X[self.mad_cols].to_numpy(dtype=float)
            med = np.nanmedian(A, axis=0)
            mad_raw = np.nanmedian(np.abs(A - med), axis=0)
            c75 = 0.67448975
            scale = mad_raw / c75
            scale = np.where((~np.isfinite(scale)) | (scale == 0), 1.0, scale)
            self.mad_median_ = med
            self.mad_scale_ = scale

        # ---- stats for sparse cols (nonzero values only) ----
        if self.sparse_cols:
            A = X[self.sparse_cols].to_numpy(dtype=float)
            A[A == 0] = np.nan  # ignore zeros when fitting
            med = np.nanmedian(A, axis=0)
            mad_raw = np.nanmedian(np.abs(A - med), axis=0)
            c75 = 0.67448975
            scale = mad_raw / c75
            scale = np.where((~np.isfinite(scale)) | (scale == 0), 1.0, scale)
            self.sparse_median_ = med
            self.sparse_scale_ = scale

        # ---- stats for IQR robust cols ----
        if self.robust_cols_:
            R = X[self.robust_cols_].to_numpy(dtype=float)
            med_r = np.nanmedian(R, axis=0)
            q25 = np.nanquantile(R, 0.25, axis=0)
            q75 = np.nanquantile(R, 0.75, axis=0)
            iqr = q75 - q25
            iqr = np.where((~np.isfinite(iqr)) | (iqr == 0), 1.0, iqr)
            self.robust_median_ = med_r
            self.robust_scale_ = iqr

        return self

    def transform(self, X):
        X = X.copy()

        # ---- MAD + arcsinh ----
        if self.mad_cols:
            A = X[self.mad_cols].to_numpy(dtype=float)
            Z = (A - self.mad_median_) / self.mad_scale_
            X[self.mad_cols] = np.arcsinh(Z)

        # ---- sparse cols: add indicator + transform nonzeros ----
        if self.sparse_cols:
            A = X[self.sparse_cols].to_numpy(dtype=float)
            for i, col in enumerate(self.sparse_cols):
                # indicator
                X[f"has_{col}"] = (A[:, i] != 0).astype(int)
                # transform only nonzeros
                nonzero_mask = A[:, i] != 0
                Z = (A[:, i] - self.sparse_median_[i]) / self.sparse_scale_[i]
                Z = np.arcsinh(Z)
                Z[~nonzero_mask] = 0  # keep zeros
                X[col] = Z

        # ---- robust cols ----
        if self.robust_cols_:
            R = X[self.robust_cols_].to_numpy(dtype=float)
            Zr = (R - self.robust_median_) / self.robust_scale_
            X[self.robust_cols_] = Zr

        return X

    def get_feature_names_out(self, input_features=None):
        # passthrough + mad + sparse (with has_) + robust
        extra = [f"has_{c}" for c in self.sparse_cols]
        return np.concatenate([self.feature_names_in_, extra])


In [ ]:
df_train = pd.read_parquet('train_fullyears.parquet')
test_2024 = pd.read_parquet('test_2024.parquet')
df_variables = pd.read_parquet('df_train_variables.parquet')
df_test_variables = pd.read_parquet('df_test_variables.parquet')
df_full_variables = pd.concat([df_variables, df_test_variables])

df_train['ExecutionTime'] = df_train['ExecutionTime'].dt.tz_localize(None)
test_2024['ExecutionTime'] = test_2024['ExecutionTime'].dt.tz_localize(None)

In [ ]:
nbeatsx_full = pd.concat([df_train[df_train['ExecutionTime'].dt.year==2023], test_2024])

# preprocess the data to merge all external variables, remove outliers, and filter out discontinue contracts
nbeatsx_full = preprocess_2023(nbeatsx_full,df_full_variables , train=True)
train_nbeatsx, test_nbeatsx = get_last_16_steps_2024(nbeatsx_full[0])

# scale the data, and transform test dataset based on the scaler of train dataset
scaler = CustomScaler(
    mad_cols=["close","high","low","day_ahead_price","price_range","q2h_price_diff"],
    sparse_cols=["imported","exported","volume","q2h_volume_diff"],
    passthrough_cols=['ExecutionTime','ID','delvery_time']
)

train_nbeatsx_s = scaler.fit_transform(train_nbeatsx)
test_nbeatsx_s = scaler.transform(test_nbeatsx)

delivery_time amounts: 56354
columns: 10
before merge: (4424871, 10)
after merge: (4424871, 26)
before remove outlier 56354
after remove outlier  56280
before continutiy check: 56280
after continutiy check: 54022


# Model Training

In [ ]:
def train_nbeatsx(
    nbeatsx_train,
    static_train,
    futr_cols,
    hist_cols,
    static_cols,
    h=16,
    input_size=24,
    dropout=0.1,
    batch_size=64,
    lr=1e-3,
    max_steps=500,
    val_check_steps=10,
    early_stop_patience_steps=5,
    freq='15min',
    val_size=12,
    stack_types=("identity","trend","seasonality","exogenous"),
    n_blocks=(2,2,2,2),
    mlp_units=((128,128),(128,128),(128,128),(128,128)),
    loss=MAE(),
    scaler_type=None,
):
    """
    Train an NBEATSx model and return the fitted NeuralForecast object.

    Parameters
    ----------
    nbeatsx_train : pd.DataFrame
        Training data formatted for NeuralForecast.
    static_train : pd.DataFrame
        Static exogenous features aligned with series IDs.
    futr_cols, hist_cols, static_cols : list[str]
        Column names for future, historical, and static exogenous variables.
    Other kwargs mirror NBEATSx/NeuralForecast hyperparameters.

    Returns
    -------
    nf : NeuralForecast
        Fitted forecaster (use nf.predict() for forecasts).
    model : NBEATSx
        The configured NBEATSx model instance.
    """

    model = NBEATSx(
        h=h,
        input_size=input_size,
        stack_types=stack_types,
        n_blocks=n_blocks,
        dropout_prob_theta=dropout,
        futr_exog_list=futr_cols,
        hist_exog_list=hist_cols,
        stat_exog_list=static_cols,
        batch_size=batch_size,
        learning_rate=lr,
        max_steps=max_steps,
        val_check_steps=val_check_steps,
        scaler_type=scaler_type,
        mlp_units=mlp_units,
        loss=loss,
    )

    nf = NeuralForecast(models=[model], freq=freq)
    nf.fit(nbeatsx_train, val_size=val_size, static_df=static_train)
    return nf, model

In [ ]:
def evaluation(df, df_raw,  median, scale):

  Z = np.sinh(df.iloc[:,-1].to_numpy(dtype=float))
  x = Z * scale + median
  df[f"y_pred"] = x
  df_result = df_raw.merge(df, on=['unique_id','ds'],how='inner')
  y_true = df_result["y"].to_numpy()
  y_pred = df_result["y_pred"].to_numpy()

  # SMAPE
  smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))
  # MAE
  mae = np.mean(np.abs(y_pred - y_true))
  # RMSE
  rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))

  print(f"smape: {smape}")
  print(f"mae: {mae}")
  print(f"rmse: {rmse}")

  return df_result


def final_metric_comparison(nbeatsx_m):
  nbeatsx_m["smape"] = (200 * np.abs(nbeatsx_m["y_pred"] - nbeatsx_m["y"]) / (np.abs(nbeatsx_m["y"]) + np.abs(nbeatsx_m["y_pred"]) + 1e-8))
  nbeatsx_m["mae"] = np.abs(nbeatsx_m["y_pred"] - nbeatsx_m["y"])
  metrics_by_id = (
      nbeatsx_m.groupby("unique_id")[["smape", "mae"]]
      .mean()
      .reset_index())
  print(f"Average SMAPE: {nbeatsx_m['smape'].mean()}%")
  print(f"Min SMAPE: {metrics_by_id['smape'].min()}%")
  print(f"Max SMAPE: {metrics_by_id['smape'].max()}%")
  print(f"Median SMAPE: {metrics_by_id['smape'].median()}%")

  print(f"Average MAE: {nbeatsx_m['mae'].mean()}")
  print(f"Min MAE: {metrics_by_id['mae'].min()}")
  print(f"Max MAE: {metrics_by_id['mae'].max()}")
  print(f"Median MAE: {metrics_by_id['mae'].median()}")
  return nbeatsx_m, metrics_by_id


In [ ]:
nbeatsx_train = add_temproal_features(train_nbeatsx_s)
nbeatsx_test = add_temproal_features(test_nbeatsx_s)

nbeatsx_train = rename_df_2023(nbeatsx_train)
nbeatsx_test = rename_df_2023(nbeatsx_test)
nbeatsx_test_actual = test_nbeatsx[['ID','ExecutionTime','close','delivery_time']].rename(columns={"ExecutionTime": "ds", "delivery_time":"unique_id", "close": "y"})

In [ ]:
hist_cols = ['high', 'low', 'volume','has_volume', 'price_range', 'q2h_price_diff', 'q2h_volume_diff', 'time_to_expiry_h', 'has_q2h_volume_diff']

futr_cols = ['time_to_expiry_h' ]

static_cols = ['day_ahead_price','load_forecast', 'solar_forecast', 'wind_forecast', 'temperature_rounded','imported', 'exported',
              'day_of_year_sin', 'day_of_year_cos', 'delivery_month_sin', 'delivery_month_cos',
               'delivery_hour_sin', 'delivery_hour_cos', 'delivery_weekday_sin', 'delivery_weekday_cos','qh_sin', 'qh_cos',
               'special_period', 'has_imported', 'has_exported']

In [ ]:
nbeatsx_X_test = nbeatsx_test.drop(columns=['y'])
y_test = nbeatsx_test_actual[['unique_id','ds','y']]
static_train = nbeatsx_train[['unique_id'] + static_cols].drop_duplicates('unique_id')
static_test  = nbeatsx_test[['unique_id'] + static_cols].drop_duplicates('unique_id')

In [ ]:
nf, model = train_nbeatsx(nbeatsx_train, static_train, futr_cols, hist_cols, static_cols,
                          h=16,
                          input_size=32,
                          dropout=0.1,
                          batch_size=64,
                          lr=1e-3,
                          max_steps=500,
                          val_check_steps=10,
                          early_stop_patience_steps=5,
                          freq='15min',
                          val_size=16,
                          stack_types=["identity","trend","seasonality","exogenous"],
                          n_blocks=[2,2,2,2],
                          mlp_units=[(128,128),(128,128),(128,128),(128,128)],
                          loss=MAE(),
                          scaler_type=None)

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_t

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=500` reached.


In [ ]:
future_df = nf.make_future_dataframe()
futr_df_predict = future_df.merge(nbeatsx_test, on=['unique_id', 'ds'], how='left')
futr_df_predict = futr_df_predict.fillna(0)
fcst_t = nf.predict(futr_df=futr_df_predict)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
df_result = evaluation(fcst_t, y_test,  median = scaler.mad_median_[0], scale = scaler.mad_scale_[0])

smape: 20.47531012073577
mae: 8.461178347926298
rmse: 13.395616645502583


In [ ]:
df_result, df_result_group = final_metric_comparison(df_result)

Average SMAPE: 20.47531012073577%
Min SMAPE: 0.2295493996099489%
Max SMAPE: 199.99999991736786%
Median SMAPE: 8.878080729489842%
Average MAE: 8.461178347926298
Min MAE: 0.13954063320201815
Max MAE: 114.12225869895943
Median MAE: 6.3508554340115975


In [ ]:
df_result

,unique_id,ds,y,NBEATSx,y_pred,smape,mae
0,2024-01-07 00:45:00,2024-01-06 20:45:00,79.20,-0.331731,79.026783,0.218948,0.173217
1,2024-01-07 00:45:00,2024-01-06 21:00:00,79.99,-0.327551,79.196219,0.997298,0.793781
2,2024-01-07 00:45:00,2024-01-06 21:15:00,79.50,-0.331007,79.056156,0.559857,0.443844
3,2024-01-07 00:45:00,2024-01-06 21:30:00,79.80,-0.336231,78.844109,1.205076,0.955891
4,2024-01-07 00:45:00,2024-01-06 21:45:00,81.05,-0.335740,78.864073,2.733877,2.185927
...,...,...,...,...,...,...,...
335275,2024-08-21 23:45:00,2024-08-20 21:45:00,71.69,-0.503336,71.840143,0.209214,0.150143
335276,2024-08-21 23:45:00,2024-08-20 22:00:00,70.93,-0.501802,71.906674,1.367540,0.976674
335277,2024-08-21 23:45:00,2024-08-20 22:15:00,63.82,-0.506865,71.686863,11.611018,7.866863
335278,2024-08-21 23:45:00,2024-08-20 22:30:00,65.52,-0.502525,71.875312,9.251134,6.355312


In [ ]:
df_result.to_parquet('nbeatsx_delivery_time_result.parquet',index=False)

# Result Analysis

In [ ]:
from typing import Dict, Literal, Optional
import plotly.express as px


def _ensure_dt(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if not np.issubdtype(df['unique_id'].dtype, np.datetime64):
        df['unique_id'] = pd.to_datetime(df['unique_id'])
    if not np.issubdtype(df['ds'].dtype, np.datetime64):
        df['ds'] = pd.to_datetime(df['ds'])
    return df


def _add_group_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['delivery_hour'] = df['unique_id'].dt.hour
    minutes = df['unique_id'].dt.minute
    df['quarter_hour'] = (minutes // 15) * 15
    df['time_to_expiry_h'] = ((df['unique_id'] - df['ds']).dt.total_seconds() / 3600.0).astype
    month = df['unique_id'].dt.month
    df['season'] = month.map(lambda x: (
        "Spring" if x in [3,4,5] else
        "Summer" if x in [6,7,8] else
        "Autumn" if x in [9,10,11] else
        "Winter"
    ))
    return df


def _prepare_long_df(models: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    dfs = []
    for name, d in models.items():
        dd = _ensure_dt(d)
        dd = _add_group_columns(dd)
        dd = dd.copy()
        dd['model'] = name
        dfs.append(dd)
    return pd.concat(dfs, ignore_index=True)

# -------------------- main plot function --------------------

def plot_model_metric_plotly(
    models: Dict[str, pd.DataFrame],
    metric: Literal['smape','mae'] = 'smape',
    groupby: Literal['delivery_hour','quarter_hour','season','time_to_expiry_h'] = 'delivery_hour',
    agg: Literal['median','mean'] = 'median',
    sort_x: bool = True,
    template: str = 'plotly_white',
    height: Optional[int] = 500,
    width: Optional[int] = None,
    markers: bool = True,
    title: Optional[str] = None,
) -> pd.DataFrame:
    """
    Plotly line plot：compare different models' performences(smape, mae) under different group(delivery_hour, quarter_hour, season, time_to_expiry)
    """
    long_df = _prepare_long_df(models)
    grp = long_df.groupby([groupby, 'model'], dropna=False)[metric]

    # choose mean or median
    if agg == 'mean':
        stats = grp.mean().reset_index(name='center')
    else:
        stats = grp.median().reset_index(name='center')

    pivot_df = stats.pivot(index=groupby, columns='model', values='center')

    category_orders = None
    if groupby == 'season' and sort_x:
        category_orders = {groupby: ['Spring','Summer','Autumn','Winter']}
    elif sort_x:
        stats = stats.sort_values(by=groupby)

    fig = px.line(
        stats,
        x=groupby,
        y='center',
        color='model',
        markers=markers,
        category_orders=category_orders,
        title=title or f"{('MEDIAN' if agg=='median' else 'MEAN')} {metric.upper()} by {groupby}",
        template=template,
    )

    fig.update_layout(
        xaxis_title=groupby,
        yaxis_title=metric,
        height=400,
        width=1000,
        legend_title_text='model',
    )

    fig.show()
    return pivot_df


In [ ]:
models = {'NBEATSx': df_result} ## you can add other model's result, but they should have same columns including y and y_pred, unique_id, ds

pivot_df = plot_model_metric_plotly(models, agg='mean', metric='smape', groupby='delivery_hour', template='plotly_white')

In [ ]:
pivot_df

model,NBEATSx
delivery_hour,
0,13.419930
1,14.026652
2,14.913956
3,16.946995
4,16.444209
5,15.139365
6,15.159683
7,15.232640
8,17.220706


In [ ]:
import nbformat, os

nb_in = "/content/Aggregated_ID_Model_Training.ipynb"
nb_tmp = "/content/tmp_clean.ipynb"
nb_out = "/content/Aggregated_ID_Model_Training.html"

# load notebook
nb = nbformat.read(nb_in, as_version=4)
# remove the problematic widget metadata
nb.metadata.pop("widgets", None)
# save a cleaned version
nbformat.write(nb, nb_tmp)

# now safely convert to HTML
os.system(f'jupyter nbconvert --to html "{nb_tmp}" --output "{nb_out}"')
print("HTML saved to:", nb_out)
